# Vectorization
Most of the data available will eventually result in vectors.

In short, we will write the vector operations in a faster way (much faster than looping through them).
Some programming languages already support this concept (another term is "array programming"), but in Python, Numpy library does the job.

This example tries to sum over array A and B while putting the result back into array A. Here is how a normal, iterative solution would look like in languages like C or Pascal: (The code is written in C)

In [ ]:
for (int i = 0; i < n; i++)
    A[i] = A[i] + B[i]

While the vectorized form look like this:

In [ ]:
A = A + B

Now, let's compare how much speedup we gain using this technique.

In [1]:
import numpy as np  # import the numpy package
import random  # to generate random numbers

In [2]:
# Scalar way
A = [random.randint(1, 1000) for _ in range(1000000)]
%timeit [x + 5 for x in A]

67.8 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
# Vectorized way
A = np.array([random.randint(1, 1000) for _ in range(1000000)])
%timeit A + 5

2.14 ms ± 85.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Here the value 5 is "broadcasted" to the whole array. For this simple operation we have ~35x speedup. Let's try a more difficult operation, the outer product.

In [4]:
import time

In [5]:
a = [random.randint(1, 1000) for _ in range(1000)]  # 1 * N
b = [random.randint(1, 1000) for _ in range(1000)]  # 1 * N

In [6]:
start = time.time()

# Scalar approach
c = np.zeros((1000, 1000))  # the result dimension is a N * N
for i in range(len(a)):
    for j in range(len(b)):
        c[i][j] += a[i] * b[j]
print(c)
    
print(f"\nThe elapsed time was {1000*(time.time() - start)} ms!")

[[ 14399.   4471.  16847. ...   8160.  15606.  13158.]
 [609840. 189360. 713520. ... 345600. 660960. 557280.]
 [ 80465.  24985.  94145. ...  45600.  87210.  73530.]
 ...
 [485331. 150699. 567843. ... 275040. 526014. 443502.]
 [739431. 229599. 865143. ... 419040. 801414. 675702.]
 [464156. 144124. 543068. ... 263040. 503064. 424152.]]

The elapsed time was 1110.7234954833984 ms!


In [7]:
start = time.time()

# Vectorized approach
outer = np.outer(a, b)
print(outer)
    
print(f"\nThe elapsed time was {1000*(time.time() - start)} ms!")

[[ 14399   4471  16847 ...   8160  15606  13158]
 [609840 189360 713520 ... 345600 660960 557280]
 [ 80465  24985  94145 ...  45600  87210  73530]
 ...
 [485331 150699 567843 ... 275040 526014 443502]
 [739431 229599 865143 ... 419040 801414 675702]
 [464156 144124 543068 ... 263040 503064 424152]]

The elapsed time was 7.622718811035156 ms!


~300x speedup!

More functions are available in numpy which will provide faster runtime. [Here](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/VectorizedOperations.html) you can find some of them.

# Array Broadcasting
Numpy tries to broadcast the lower-dimension matrix along a new dimension when performing operations on matrices with unequal shapes.  Let's look at an example:

In [8]:
a = np.array([[1, 2], [4, 5], [6, 7]])
a.shape

(3, 2)

In [9]:
b = np.array([[10, 10]])
b.shape

(1, 2)

In [10]:
# Element-wise multiplication
a * b, (a * b).shape

(array([[10, 20],
        [40, 50],
        [60, 70]]),
 (3, 2))

Here the second matrix was treated as if it has a dimension of (3, 2). Note that Numpy does not create the duplicated matrix as it would be a waste of time and space.

You can't broadcast **any** matrix you want. There are some rules to broadcasting which will be discussed later on. For example, the following operation will not be performed.

In [11]:
np.array([1, 2]) * np.array([0, 1, 2])

ValueError: operands could not be broadcast together with shapes (2,) (3,) 

### Broadcasting rules
You can't broadcast arbitrary combinations of array shapes together. For instance, adding a (2, 5) matrix with a (6, 9) matrix will result in a **ValueError**.

To determine whether two matrices are broadcast-compatible, Numpy starts from the rightmost dimension and moves to the right, constantly checking these two rules (one of them will suffice). If none of the rules apply for a dimension, then the broadcasting cannot be done. Note that missing dimensions are treated as if they are 1.
 
The rules:
1. Both dimensions are equal, or
2. One of them is 1

The resulting shape would be the equal value (when both are equal) or the maximum value (when one of them is 1) for each dimension.

Here are some examples:
1. (5, 3, 2) and (2) => (5, 3, 2)
2. (1, 2, 4) and (2, 4) => (1, 2, 4)
3. (6, 1, 7, 3) and (6, 1, 3) => (6, 6, 7, 3)
4. (6, 5, 3) and (2) => not compatible!
5. (7, 2, 3) and (3, 3) => not compatible!

In [12]:
a = np.zeros((6, 1, 7, 3))
b = np.ones((6, 1, 3))
(a * b).shape

(6, 6, 7, 3)

In [13]:
a = np.zeros((7, 2, 3))
b = np.ones((3, 3))
(a * b).shape

ValueError: operands could not be broadcast together with shapes (7,2,3) (3,3) 